In [1]:
import json
import os
import torch
from transformers import BertTokenizer, BertModel, AutoModelForCausalLM, AutoTokenizer
import os.path as osp

/home/yiyao_yang/anaconda3/envs/graphgpt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llama_path = '/data/LPJ/Llama-2-7b-chat-hf'
# llama_path = '/data/LPJ/Llama-2-7b-hf'
tokenizer = AutoTokenizer.from_pretrained(llama_path)
model = AutoModelForCausalLM.from_pretrained(llama_path, torch_dtype=torch.bfloat16, device_map=0)

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.88s/it]


In [33]:
test = '''Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{'id': 0, 'content': 'data_a'}, {'id': 1, 'content': 'iSAOUT'}, {'id': 2, 'content': 'out8'}, {'id': 3, 'content': 'led_0'}, {'id': 4, 'content': 'op1'}, {'id': 5, 'content': 'ripple_adder_32'}], \nedge_attrs: [], \nconnectivity: [[1, 4], [5, 4]]\n, please find which nodes point to ripple_adder_32 node, and ripple_adder_32 node points to which nodes, 
you should use a template like below and just output content in this template, don't output anything else, like Explanation:
{which nodes} points to {above node}, {above node} points to {which nodes}
'''
test = '''Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{'id': 0, 'content': 'data_a'}, {'id': 1, 'content': 'iSAOUT'}, {'id': 2, 'content': 'out8'}, {'id': 3, 'content': 'led_0'}, {'id': 4, 'content': 'op1'}, {'id': 5, 'content': 'ripple_adder_32'}], \nedge_attrs: [], \nconnectivity: [[1, 4], [5, 4]]\n, please find which nodes point to ripple_adder_32 node, and ripple_adder_32 node points to which nodes, 
you should just output content in this template, don't output anything else, like Explanation:
{which nodes} points to {above node}, {above node} points to {which nodes}
'''
inputs = tokenizer(test, return_tensors="pt")


In [34]:
for key, value in inputs.items():
    inputs[key] = value.to('cuda:0')

In [35]:

outputs = model.generate(**inputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Given a submodules interconnection graph: 
<graph>
, 
nodes: [{'id': 0, 'content': 'data_a'}, {'id': 1, 'content': 'iSAOUT'}, {'id': 2, 'content': 'out8'}, {'id': 3, 'content': 'led_0'}, {'id': 4, 'content': 'op1'}, {'id': 5, 'content': 'ripple_adder_32'}], 
edge_attrs: [], 
connectivity: [[1, 4], [5, 4]]
, please find which nodes point to ripple_adder_32 node, and ripple_adder_32 node points to which nodes, 
you should just output content in this template, don't output anything else, like Explanation:
{which nodes} points to {above node}, {above node} points to {which nodes}

For example, if the answer is:
{0, 1, 2} points to {ripple_adder_32}, ripple_adder_32 points to {0, 1, 2}

Then, the output should be:
{0, 1, 2} points to {ripple_adder_32}, ripple_adder_32 points to {0, 1, 2}

Please let me know if you have any questions or need further clarification.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import os
import numpy as np

def seed_torch(seed=1029):
	random.seed(seed)
	os.environ['PYTHONHASHSEED'] = str(seed) # 为了禁止hash随机化，使得实验可复现
	np.random.seed(seed)
	torch.manual_seed(seed)
	torch.cuda.manual_seed(seed)
	torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
	torch.backends.cudnn.benchmark = False
	torch.backends.cudnn.deterministic = True

seed_torch()

# 定义一个网络
class net(nn.Module):
    def __init__(self, num_class=10):
        super(net, self).__init__()
        self.pool1 = nn.AvgPool1d(2)
        self.bn1 = nn.BatchNorm1d(3)
        self.fc1 = nn.Linear(12, 4)

    
    def forward(self, x):
        x = self.pool1(x)
        x = self.bn1(x)
        x = x.reshape(x.size(0), -1)
        
        x = x.detach()  # 将非叶子节点剥离成叶子节点 x.requires_grad = False x.grad_fn=None

        y = self.fc1(x)

        return y


# 定义网络
model = net()

# 定义loss
loss_fn = nn.CrossEntropyLoss()

# 定义优化器
optimizer = optim.SGD(model.parameters(), lr=1e-2)

# 定义训练数据
x = torch.randn((3, 3, 8))

# 训练前叶子结点 bn1.weight 的参数情况
print(model.bn1.weight.requires_grad)
print(model.bn1.weight.grad)

# 训练前叶子结点 fc1.weight 的参数情况
print(model.fc1.weight.requires_grad)
print(model.fc1.weight.grad)

output = model(x)
target = torch.tensor([1, 1, 1])
loss = loss_fn(output, target)

loss.backward()

# 训练后叶子结点 bn1.weight 的参数情况
print(model.bn1.weight.requires_grad)
print(model.bn1.weight.grad)

# 训练后叶子结点 fc1.weight 的参数情况
print(model.fc1.weight.requires_grad)
print(model.fc1.weight.grad)


In [2]:
folder_path = '/data/LPJ/ICML25/GraphCoder/test_mkdir/son_dir'
os.makedirs(folder_path, exist_ok=True)
print(f"已创建文件夹及其子文件夹：{folder_path}")


已创建文件夹及其子文件夹：/data/LPJ/ICML25/GraphCoder/test_mkdir/son_dir


In [2]:
bert_tokenizer = BertTokenizer.from_pretrained(
        '/data/LPJ/bert/bert-L12-H128-uncased',
        model_max_length=25,
        padding_side="right")


In [3]:
raw_node_attri = ['aaaaaa', 'i am your father and you are my son']

In [4]:
rslt = bert_tokenizer(
            raw_node_attri,
            return_tensors="pt",
            padding="longest",
            max_length=bert_tokenizer.model_max_length,
            truncation=True
        )

In [5]:
rslt

{'input_ids': tensor([[  101, 13360, 11057,  2050,   102,     0,     0,     0,     0,     0,
             0],
        [  101,  1045,  2572,  2115,  2269,  1998,  2017,  2024,  2026,  2365,
           102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [13]:
**rslt

SyntaxError: invalid syntax (1093260329.py, line 1)

In [9]:
bert_tokenizer.pad_token
bert_tokenizer.pad_token_id
bert_tokenizer.eos_token
bert_tokenizer.eos_token_id


In [11]:
bert_model = BertModel.from_pretrained('/data/LPJ/bert/bert-L12-H128-uncased', torch_dtype=torch.bfloat16, device_map='cpu')


/home/yiyao_yang/anaconda3/envs/graphgpt/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [16]:
rep = bert_model(**rslt)


BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-1.0312,  0.3008, -0.7188,  ...,  0.0064, -0.4648, -0.2490],
         [ 0.9688,  0.0889, -1.5938,  ..., -1.3750, -1.2656,  0.4629],
         [ 1.0312, -0.9570, -1.5781,  ..., -1.0781, -1.2734, -0.1660],
         ...,
         [-0.3438,  0.1875, -0.5508,  ...,  0.0659, -0.0747,  0.2158],
         [-0.1426,  0.1514, -0.8008,  ...,  0.0449, -0.1279,  0.2773],
         [ 0.6211,  0.3379, -1.4375,  ..., -1.1484, -1.0625,  0.2852]],

        [[-0.9727,  0.1641,  1.3516,  ..., -0.9727, -0.2363, -0.7422],
         [ 0.7383, -0.8555,  0.1631,  ..., -1.5000,  1.0312,  0.2041],
         [ 0.5195, -0.2754, -0.3008,  ..., -0.8789,  1.4922, -0.9648],
         ...,
         [ 0.5273, -1.1719,  0.5039,  ..., -0.2432,  0.2021, -0.3262],
         [-0.0352,  0.0500,  1.1016,  ..., -1.3906, -0.7852,  0.7070],
         [-1.1328, -0.5664,  0.3320,  ..., -0.3438, -0.3457, -0.6289]]],
       dtype=torch.bfloat16, grad_fn=<NativeLayerNorm

In [19]:
rep.pooler_output.shape

torch.Size([2, 128])